In [21]:
import polars as pl
import plotly.express as px
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [22]:
credit_discrete_map = {False: 'blue', True:'red'}

In [23]:
df = pl.read_csv('Data/data.csv', has_header=True, skip_rows=5, separator=';', try_parse_dates=True)
df = (
    df.rename({'Data valuta':'Date'})
    .with_columns([
        pl.coalesce([pl.col('Entrate'), pl.col('Uscite')]).alias('Amount'),
        pl.col('Entrate').is_not_null().alias('Credit'),
        (pl.col('Causale') == 'EMOLUMENTI').alias('Salary'),
        pl.col('Date').dt.year().alias('Year'),
        pl.col('Date').dt.month().alias('Month')
    ])
    .select(['Date', 'Year', 'Month', 'Amount', 'Credit', 'Salary'])
)
df = df.sort('Date')

In [25]:
date_start = widgets.DatePicker(value=df['Date'].min(), description='Date start')
date_end = widgets.DatePicker(value=df['Date'].max(), description='Date end')

In [26]:
display(widgets.VBox([date_start, date_end]))

In [ ]:
def indicators(start, end):
    temp = df.filter((pl.col('Date')>=start) & (pl.col('Date')<=end))
    indicators = make_subplots(
        2, 2,
        specs=[[{"type": "domain"}, {"type": "domain"}], [{"type": "domain"}, {"type": "domain"}]]
    )
    indicator_font_size = 50
    indicator_title_size = 27
    indicators.add_trace(
        go.Indicator(
            mode='number+delta',
            value=temp.filter(pl.col('Credit'))['Amount'].sum() - temp.filter(~pl.col('Credit'))['Amount'].sum(),
            delta={'reference':0.1, 'relative': True, 'valueformat':',.0%'},
            number={'prefix':'€', 'valueformat':',.2f', 'font':{'size':indicator_font_size}},
            title={"text": 'Netto transazioni', 'font': {'size': indicator_title_size}}
        ),
        row=1,
        col=1
    )
    indicators.add_trace(
        go.Indicator(
            mode='number',
            value=temp['Amount'].len(),
            number={'valueformat':',.0f', 'font':{'size':indicator_font_size}},
            title={"text": '# transazioni', 'font': {'size': indicator_title_size}}
        ),
        row=1,
        col=2
    )
    indicators.add_trace(
        go.Indicator(
            mode='number',
            value=temp.filter(pl.col('Credit'))['Amount'].max(),
            number={'prefix':'€', 'valueformat':',.2f', 'font':{'size':indicator_font_size}},
            title={"text": 'Max in entrata', 'font': {'size': indicator_title_size}}
        ),
        row=2,
        col=1
    )
    indicators.add_trace(
        go.Indicator(
            mode='number',
            value=temp.filter(~pl.col('Credit'))['Amount'].max(),
            number={'prefix':'€', 'valueformat':',.2f', 'font':{'size':indicator_font_size}},
            title={"text": 'Max in uscita', 'font': {'size': indicator_title_size}}
        ),
        row=2,
        col=2
    )
    indicators.update_layout(
        uniformtext_mode='hide',
        uniformtext_minsize=11,
        width=1000
    )
    indicators.show()
indicators = widgets.interactive_output(indicators, {'start': date_start, 'end': date_end})
display(indicators)

In [ ]:
def piecharts(start, end):
    temp = (
        df.filter((pl.col('Date')>=start) & (pl.col('Date')<=end))
        .groupby('Credit').agg(
            pl.col('Amount').sum().alias('Amount'),
            pl.col('Date').count().alias('Count')
        ).select('Credit', pl.col('Amount')/pl.col('Amount').sum(), pl.col('Count')/pl.col('Count').sum())
    )
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]], subplot_titles=("Amount", "Count"))
    fig.add_trace(
        go.Pie(
            labels = temp['Credit'].to_list(),
            values = temp['Amount'].to_list(),
            hole = 0.4
        ),
        row=1,
        col=1
    )
    fig.add_trace(
        go.Pie(
            labels = temp['Credit'].to_list(),
            values = temp['Count'].to_list(),
            hole = 0.4
        ),
        row=1,
        col=2
    )
    fig.update_layout(height=500, width=1000, legend={'title':'Credit'})
    fig.show()
piecharts = widgets.interactive_output(piecharts, {'start': date_start, 'end': date_end})
display(piecharts)

In [ ]:
def histplot(start, end):
    fig = px.histogram(
        data_frame = df.filter((pl.col('Date')>=start) & (pl.col('Date')<=end)).to_pandas(),
        x = 'Amount',
        histnorm = 'percent',
        title = 'Amount distribution',
        facet_row = 'Credit',
        color = 'Credit',
        color_discrete_map = credit_discrete_map,
        nbins = 50,
        height = 800
    )
    fig.show()
histplot = widgets.interactive_output(histplot, {'start': date_start, 'end': date_end})
display(histplot)

In [ ]:
def scatter(start, end):
    fig = px.scatter(
        data_frame = df.filter((pl.col('Date')>=start) & (pl.col('Date')<=end)).to_pandas(),
        x = 'Date',
        y = 'Amount',
        color = 'Credit',
        color_discrete_map=credit_discrete_map
    )
    fig.show()
scatter = widgets.interactive_output(scatter, {'start': date_start, 'end': date_end})
display(scatter)